In [1]:
import os
import numpy as np
import pandas as pd
import random
import pydicom as dicom
import cv2

In [2]:
import os
os.chdir('/Users/raj/Data_science/Melanoma_Classification')

df = pd.read_csv('train.csv')
df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [3]:
df_count = pd.read_csv('Melanoma_patient_info.csv')
df_count.head()


,Patient_id,Total_image,Melanoma_image,Melanoma_label
0,IP_7279968,115,0,0
1,IP_3075186,24,0,0
2,IP_2842074,5,0,0
3,IP_6890425,22,1,1
4,IP_8723313,20,0,0


In [4]:
seed=5
seed=5
s1 = df.target[df.target.eq(1)].index
s0 = df.target[df.target.eq(0)].sample(len(s1),random_state=seed).index
print(len(s1),len(s0))

dff= df.loc[s0.union(s1)]
dff.head()

584 584


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
91,ISIC_0149568,IP_0962375,female,55.0,upper extremity,melanoma,malignant,1
158,ISIC_0167093,IP_8029095,female,45.0,upper extremity,unknown,benign,0
192,ISIC_0175530,IP_4479736,female,25.0,lower extremity,unknown,benign,0
235,ISIC_0188432,IP_0135517,female,50.0,upper extremity,melanoma,malignant,1
297,ISIC_0202680,IP_0161742,female,40.0,torso,unknown,benign,0


In [5]:
dict = {'Patient_id': dff.patient_id, 'image_name': dff.image_name, 'image_label':dff.target,'df_index':dff.index}   
df_equal= pd.DataFrame(dict)
df_equal.to_csv("Selected_equal_patient.csv",index=False)




In [6]:
# Select two non-melanoma images from each patient and all melanoma images if available

seed=5
select_non_melanoma=2

patient_id_nonmelanoma=[]
image_name_nonmelanoma=[]
image_name_melanoma=[]
image_nonmelanoma_label=[]
image_melanoma_label=[]
patient_id_melanoma=[]
index_melanoma=[]
index_nonmelanoma=[]

for i in df_count.Patient_id:
    #print(i,"hi")
    label=df_count[df_count.Patient_id==i].Melanoma_label.iloc[0]
    count=df_count[df_count.Patient_id==i].Total_image.iloc[0]
    #print(i,label,count)
    if (label==0):
        if (count>select_non_melanoma):
            s0 = df[df.patient_id==i].sample(select_non_melanoma,random_state=seed).index
            #print(s0)
            images=df.loc[s0].image_name
            #print(images)
            for k in range(len(images)):
                image_name_nonmelanoma.append(images.iloc[k])
                image_nonmelanoma_label.append(0)  
                patient_id_nonmelanoma.append(i)
                index_nonmelanoma.append(s0[k])
        else:
            s0 = df[df.patient_id==i].index
            images=df.loc[s0].image_name
            for k in range(len(images)):
                image_name_nonmelanoma.append(images.iloc[k])
                image_nonmelanoma_label.append(0)
                patient_id_nonmelanoma.append(i)
                index_nonmelanoma.append(s0[k])
             
            
    else:
        
        df_id=df[df.patient_id==i]
        countA=len(df_id.target[df_id.target.eq(0)])
        #print(i,label,count,countA)
        if (countA>select_non_melanoma):
            s0 = df_id.target[df_id.target.eq(0)].sample(select_non_melanoma,random_state=seed).index
            images=df.loc[s0].image_name
            for k in range(len(images)):
                image_name_melanoma.append(images.iloc[k])
                image_melanoma_label.append(0)
                patient_id_melanoma.append(i)
                index_melanoma.append(s0[k])
        else:
            s0 = df_id.target[df_id.target.eq(0)].index
            images=df.loc[s0].image_name
            for k in range(len(images)):
                image_name_melanoma.append(images.iloc[k])
                image_melanoma_label.append(0)
                patient_id_melanoma.append(i)
                index_melanoma.append(s0[k])
                

        s1=df_id.target[df_id.target.eq(1)].index
        imagesA=df.loc[s1].image_name
        
        for k in range(len(s1)):
            image_name_melanoma.append(imagesA.iloc[k])
            image_melanoma_label.append(1)
            patient_id_melanoma.append(i)
            index_melanoma.append(s1[k])

            

In [7]:
#Selected images name from melanoma patients are stored in data frame df_melanoma. For non-melanoma patients, images are stored in df_nonmelanoma.

dict = {'Patient_id': patient_id_melanoma, 'image_name': image_name_melanoma, 'image_label':image_melanoma_label,'df_index':index_melanoma}   
df_melanoma = pd.DataFrame(dict)

dictA = {'Patient_id': patient_id_nonmelanoma, 'image_name': image_name_nonmelanoma, 'image_label':image_nonmelanoma_label,'df_index':index_nonmelanoma}   
df_nonmelanoma = pd.DataFrame(dictA)

df_melanoma.to_csv("Selected_melanoma_patient_info2N.csv",index=False)
df_nonmelanoma.to_csv("Selected_nonmelanoma_patient_info2N.csv",index=False)

In [8]:
#Image scaling   
def imgResize(a1, size):
    width= a1.shape[1]
    height = a1.shape[0]
    #ratio = width / height
    new_height = size
    new_width=size
    
    dim=(new_width,new_height)

    #resized_img = resize(a1, dim, anti_aliasing=True)
    resized_img = cv2.resize(a1, dim, interpolation = cv2.INTER_AREA)

    return resized_img

#read image file
def dicom_to_array(filename):
    d = dicom.read_file(filename)
    a = d.pixel_array
    return a,d

In [9]:
df_equal.image_name

#for i in range(df_equal.shape[0]):
#    image_name=df_equal.image_name[i]

91       ISIC_0149568
158      ISIC_0167093
192      ISIC_0175530
235      ISIC_0188432
297      ISIC_0202680
             ...     
32977    ISIC_9956744
33000    ISIC_9963177
33014    ISIC_9967383
33050    ISIC_9978107
33117    ISIC_9998682
Name: image_name, Length: 1168, dtype: object

In [10]:
df_equal.image_name.iloc[0]

'ISIC_0149568'

In [11]:
#Resize equally selected images to 32 by 32, 128 by 128 and 512 by 512. Save the resized images.
import os
os.chdir('/Volumes/RAJ_256/Melanoma1')

size=32

for i in range(df_equal.shape[0]):
    image_name=df_equal.image_name.iloc[i]
    image_path ='train/'+image_name +'.dcm'
    a1,d = dicom_to_array(image_path)   
    resized_img=imgResize(a1,size) 
    
    save_path='Size32/'+'S32_'+image_name +'.png'
    cv2.imwrite(save_path, resized_img)

size=128

for i in range(df_equal.shape[0]):
    image_name=df_equal.image_name.iloc[i]
    image_path ='train/'+image_name +'.dcm'
    a1,d = dicom_to_array(image_path)   
    resized_img=imgResize(a1,size) 
    
    save_path='Size128/'+'S128_'+image_name +'.png'
    cv2.imwrite(save_path, resized_img)

size=512

for i in range(df_equal.shape[0]):
    image_name=df_equal.image_name.iloc[i]
    image_path ='train/'+image_name +'.dcm'
    a1,d = dicom_to_array(image_path)  
    resized_img=imgResize(a1,size) 
    
    save_path='Size512/'+'S512_'+image_name +'.png'
    cv2.imwrite(save_path, resized_img)


    

In [16]:
#Resize selected melanoma images to 32 by 32, 128 by 128 and 512 by 512. Save the resized images.

size=32

for i in range(df_melanoma.shape[0]):
    if (df_melanoma.image_label.iloc[i]==0):
        image_name=df_melanoma.image_name.iloc[i]
        image_path ='train/'+image_name +'.dcm'
        a1,d = dicom_to_array(image_path)   
        resized_img=imgResize(a1,size) 
    
        save_path='Size32/'+'S32_'+image_name +'.png'
        cv2.imwrite(save_path, resized_img)


size=128

for i in range(df_melanoma.shape[0]):
    if (df_melanoma.image_label.iloc[i]==0):
        image_name=df_melanoma.image_name.iloc[i]
        image_path ='train/'+image_name +'.dcm'
        a1,d = dicom_to_array(image_path)   
        resized_img=imgResize(a1,size) 
    
        save_path='Size128/'+'S128_'+image_name +'.png'
        cv2.imwrite(save_path, resized_img)


size=512

for i in range(df_melanoma.shape[0]):
    if (df_melanoma.image_label.iloc[i]==0):
        image_name=df_melanoma.image_name.iloc[i]
        image_path ='train/'+image_name +'.dcm'
        a1,d = dicom_to_array(image_path)  
        resized_img=imgResize(a1,size) 
    
        save_path='Size512/'+'S512_'+image_name +'.png'
        cv2.imwrite(save_path, resized_img)


In [17]:
size=32

for i in range(df_nonmelanoma.shape[0]):
    image_name=df_nonmelanoma.image_name.iloc[i]
    image_path ='train/'+image_name +'.dcm'
    a1,d = dicom_to_array(image_path)   
    resized_img=imgResize(a1,size) 
    
    save_path='Size32/'+'S32_'+image_name +'.png'
    cv2.imwrite(save_path, resized_img)

size=128

for i in range(df_nonmelanoma.shape[0]):
    image_name=df_nonmelanoma.image_name.iloc[i]
    image_path ='train/'+image_name +'.dcm'
    a1,d = dicom_to_array(image_path)
    resized_img=imgResize(a1,size) 
    
    save_path='Size128/'+'S128_'+image_name +'.png'
    cv2.imwrite(save_path, resized_img)

size=512

for i in range(df_nonmelanoma.shape[0]):
    image_name=df_nonmelanoma.image_name.iloc[i]
    image_path ='train/'+image_name +'.dcm'
    a1,d = dicom_to_array(image_path)   
    resized_img=imgResize(a1,size) 
    
    save_path='Size512/'+'S512_'+image_name +'.png'
    cv2.imwrite(save_path, resized_img)



In [18]:
df_melanoma.iloc[20:25]

,Patient_id,image_name,image_label,df_index
20,IP_1273286,ISIC_6251753,1,20547
21,IP_0135517,ISIC_3686800,0,11964
22,IP_0135517,ISIC_0940521,0,2811
23,IP_0135517,ISIC_0188432,1,235
24,IP_1139701,ISIC_4502605,0,14703
